In [1]:
import matplotlib.pyplot as plt
import os
import cv2
import numpy as np
from PIL import Image
from AB3DMOT_libs.kitti_utils import read_label, compute_box_3d, draw_projected_box3d, Calibration, Object3d

# Generate a video

In [2]:
seqs = ["0011"]
seq = seqs[0]
result_dir = "results/pointrcnn_Car_test/trk_image_vis/%s/"%seq
num_imgs = len(os.listdir(result_dir))
imgs = []
for img_idx in range(num_imgs):
    img_path = os.path.join(result_dir, '%06d.jpg'%img_idx)
    imgs.append(cv2.imread(img_path))
video = cv2.VideoWriter("%s.avi"%seq, cv2.VideoWriter_fourcc(*'XVID'), 10, (1242,374))
for image in imgs:
   # plt.imshow(image)
    video.write(image)
video.release()

# Visualize detection result
Load detection results from "data/KITTI/result_sha/seq" and images from "data_root/image_02/seq". Save visualization results in "./results/result_sha/det_image_vis/seq" 

In [71]:
def visdet(result_sha, data_root, seq):
    width = 1242
    height = 374
    def show_image_with_boxes(img, objects_res, object_gt, calib, save_path, height_threshold=0):
        img2 = np.copy(img) 
        # for each object, compute the bouding box in 2D image. Use the same color for objects of same obj.id. 
        for obj in objects_res:
            box3d_pts_2d, _ = compute_box_3d(obj, calib.P) # calib.P is the projection matrix from camera coord to image coord
            color_tmp = tuple([255,0,0])
            img2 = draw_projected_box3d(img2, box3d_pts_2d, color=color_tmp)
            text = 'ID: %d' % obj.id
            if box3d_pts_2d is not None:
                img2 = cv2.putText(img2, text, (int(box3d_pts_2d[4, 0]), int(box3d_pts_2d[4, 1]) - 8), cv2.FONT_HERSHEY_TRIPLEX, 0.5, color=color_tmp) 

        img = Image.fromarray(img2)
        img = img.resize((width, height))
        img.save(save_path)
    image_dir = os.path.join(data_root, 'image_02/%s' % seq)
    calib_file = os.path.join(data_root, 'calib/%s.txt' % seq) 

    save_3d_bbox_dir = os.path.join('./results',result_sha,'det_image_vis/%s' % seq)
    print(save_3d_bbox_dir)
    if not os.path.exists(save_3d_bbox_dir):
        os.mkdir(save_3d_bbox_dir)

    seq_file = os.path.join('data/KITTI', result_sha, "%s.txt"%seq)
    seq_dets = np.loadtxt(seq_file, delimiter=',')
    if len(seq_dets.shape) == 1: seq_dets = np.expand_dims(seq_dets, axis=0) 	
    if seq_dets.shape[1] == 0:
        return
    calib_tmp = Calibration(calib_file)
    # loop over frame
    min_frame, max_frame = int(seq_dets[:, 0].min()), int(seq_dets[:, 0].max())
    print(min_frame, max_frame)
    for frame in range(min_frame, max_frame + 1):

        img_tmp = np.array(Image.open(os.path.join(image_dir,"%06d.png"%frame)))
        # logging
        print_str = 'processing %s:  %d/%d   \r' % (seq,  frame, max_frame)
        print(print_str)

        dets = seq_dets[seq_dets[:,0] == frame]            # h, w, l, x, y, z, theta in camera coordinate follwing KITTI convention
        objects = []
        for i, det in enumerate(dets):
            str_to_srite = '%s -1 -1 %f %f %f %f %f %f %f %f %f %f %f %f %f %d\n' % ("Car", det[14],
                    det[2], det[3], det[4], det[5], 
                    det[7], det[8], det[9], det[10], det[11], det[12], det[13], det[-1], 1)
            objects.append(Object3d(str_to_srite))
        save_path_tmp = os.path.join(save_3d_bbox_dir, '%06d.jpg' % frame)
        show_image_with_boxes(img_tmp, objects, [], calib_tmp, save_path_tmp)

In [72]:
result_sha = "pointrcnn_Car_test"
data_root = './data/KITTI/resources/testing'
seq = "0003"
visdet(result_sha, data_root, seq)

./results/pointrcnn_Car_test/det_image_vis/0003
0 256
processing 0003:  0/256   
processing 0003:  1/256   
processing 0003:  2/256   
processing 0003:  3/256   
processing 0003:  4/256   
processing 0003:  5/256   
processing 0003:  6/256   
processing 0003:  7/256   
processing 0003:  8/256   
processing 0003:  9/256   
processing 0003:  10/256   
processing 0003:  11/256   
processing 0003:  12/256   
processing 0003:  13/256   
processing 0003:  14/256   
processing 0003:  15/256   
processing 0003:  16/256   
processing 0003:  17/256   
processing 0003:  18/256   
processing 0003:  19/256   
processing 0003:  20/256   
processing 0003:  21/256   
processing 0003:  22/256   
processing 0003:  23/256   
processing 0003:  24/256   
processing 0003:  25/256   
processing 0003:  26/256   
processing 0003:  27/256   
processing 0003:  28/256   
processing 0003:  29/256   
processing 0003:  30/256   
processing 0003:  31/256   
processing 0003:  32/256   
processing 0003:  33/256   
proc